In [2]:
import numpy as np
import pandas as pd
import os
from basic_function import get_train_log, choose_logs_in_train_and_test, get_root_path
from collections import OrderedDict
from sklearn.decomposition import NMF

In [2]:
name_list = ["user_id","seller_id"]

In [3]:
logs = get_train_log(None)
logs = logs[logs["action_type"] == 2][["user_id", "seller_id"]]

In [4]:
# logs = choose_logs_in_train_and_test(logs=logs, entity="user_id")

In [5]:
replace = OrderedDict({name_list[i]: OrderedDict(
    zip(set(logs.loc[:, name_list[i]].values), list(range(len(set(logs.loc[:, name_list[i]].values)))))) for i in range(len(name_list))})

In [6]:
for i in range(len(name_list)):
    logs[name_list[i]] = logs[name_list[i]].map(replace[name_list[i]])

In [7]:
logs[logs["seller_id"] == 4995]

,user_id,seller_id


In [8]:
a = logs.groupby(["user_id", "seller_id"]).size().reset_index(name='counts')

In [9]:
data = a.values

In [11]:
shape = (len(set(logs["user_id"])), len(set(logs["seller_id"])))

In [12]:
matrix = np.zeros(shape,dtype=np.int8)

In [14]:
for user, item, rating in data:
    matrix[user][item] = rating # Convert to 0-based index.

In [17]:
matrix[0].max()

3

In [25]:
model = NMF(n_components=128,init='nndsvd',solver="cd",  random_state=0, verbose=True)

In [26]:
x = model.fit_transform(matrix)

violation: 1.0
violation: 0.12538658587673746
violation: 0.05118823441948356
violation: 0.03107449957285507
violation: 0.017956235131335336
violation: 0.010062389003773656
violation: 0.007396914053439068
violation: 0.005945269928684492
violation: 0.0037806247449645495
violation: 0.00242892530216535
violation: 0.0018479221420958215
violation: 0.0015429228446343219
violation: 0.0013655669683721844
violation: 0.0012611743037637545
violation: 0.0011977031408388816
violation: 0.0011570436566337727
violation: 0.001132131963364019
violation: 0.0011057897761792453
violation: 0.0010774835861777185
violation: 0.0010378088074191796
violation: 0.0009839995644523583
violation: 0.0009183972532533445
violation: 0.0008696866710489864
violation: 0.0008354487205566968
violation: 0.0007946851707431884
violation: 0.0007443771930397123
violation: 0.000692270916723232
violation: 0.0006405450451171939
violation: 0.0005917069378191303
violation: 0.0005467627312572113
violation: 0.0005043218794249318
violation

In [27]:
model.components_.shape

(128, 4995)

In [36]:
pd.DataFrame(data=x, index=replace["user_id"].keys(), columns=range(x.shape[1])).to_csv(
        os.path.join(get_root_path(), "feature_vectors", "user_mf.csv"),
        float_format='%.4f',
        index_label="index")

In [38]:
pd.DataFrame(data=model.components_.T, index=replace["seller_id"].keys(), columns=range(model.components_.shape[0])).to_csv(
        os.path.join(get_root_path(), "feature_vectors", "seller_mf.csv"),
        float_format='%.4f',
        index_label="index")